In [19]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [20]:
import numpy as np
import pandas as pd

In [21]:
import datetime as dt
from dateutil.relativedelta import relativedelta
from scipy import stats
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [36]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func

In [23]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [24]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [25]:
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [26]:
# Save references to each table
measurement=Base.classes.measurement
station=Base.classes.station

AttributeError: measurement

In [27]:
# Create our session (link) from Python to the DB
session=Session(engine)

# Exploratory Climate Analysis

In [28]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.

# Calculate the date 1 year ago from the last data point in the database.
last_measurement_data_point = session.query(measurement.date).order_by(measurement.date.desc()).first()
(latest_date, ) = last_measurement_data_point
latest_date = dt.datetime.strptime(latest_date, '%Y-%m-%d')
latest_date = latest_date.date()
date_year_ago = latest_date - relativedelta(years=1)

# Perform a query to retrieve the data and precipitation scores.
data_from_12 = session.query(measurement.date, measurement.prcp).filter(measurement.date >= date_year_ago).all()

# Save the query results as a Pandas DataFrame.
last_12_data_df = pd.DataFrame(data_from_12, columns=['date', 'prcp'])

# Drop null values (if any).
last_12_data_df = last_12_data_df.dropna(how="any")

# Set the index to the date column.
last_12_data_df = last_12_data_df.set_index("date")
print(last_12_data_df)

# Sort the dataframe by date.
last_12_data_df = last_12_data_df.sort_values(by=['date'])

# Use pandas plotting with matplotlib to plot the data.

# Plot it.
plot = last_12_data_df.plot(kind="bar", figsize=(10, 5), width=15)

plot.legend(["Amount of Precipitation (Inches)"], loc="upper right")
plot.set_xlabel("Date")
plot.set_ylabel("Amount of Precipitation (Inches)")
plot.set_title(f"Amount of precipitation (inches) from {date_year_ago} to {latest_date}")

plt.tick_params(
    axis='x',         
    which='both',    
    bottom=False,      
    top=False,        
    labelbottom=False)

# Tighten the graph up so it displays nicely.
plt.tight_layout()

# Add grid lines.
plot.grid(zorder=1)

# Save figure
plt.savefig(f".precipitation_data_from_{date_year_ago}_to_{latest_date}.png", bbox_inches='tight')

# Show graph.
plt.show()

NameError: name 'measurement' is not defined

In [29]:
# Use Pandas to calcualte the summary statistics for the precipitation data
last_12_data_df.describe()

NameError: name 'last_12_data_df' is not defined

In [30]:
# Design a query to show how many stations are available in this dataset?

# Design a query to show how many stations are available in this dataset?
stationcount=session.query(station.id).count()

print(f"There are {stationcount} stations in this dataset.")

NameError: name 'station' is not defined

In [31]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [32]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
sel= [measurement.station, func.count(measurement.id)]

mostactive = session.query(*sel).group_by(measurement.station).order_by(func.count(measurement.id).desc()).all()

for record in mostactive:
    (station, count) = record
    print(
        f"One of the most active stations is {station} which was found {count} in the database.")

NameError: name 'measurement' is not defined

In [33]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
popstationdata = session.query(measurement.date, measurement.tobs).filter(measurement.station == "USC00519281").filter(measurement.date >= oneyearago).all()

#first 10
popstationdata[:10]

NameError: name 'measurement' is not defined

In [34]:
popstationdata_df = pd.DataFrame(popstationdata)
popstationdata_df.plot(kind="hist", bins=12)

NameError: name 'popstationdata' is not defined

## Bonus Challenge Assignment

In [35]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

NameError: name 'Measurement' is not defined

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
